In [ ]:
import io
import numpy as np
import cv2
import os
import torch
import PIL
import ultralytics
from ultralytics import YOLO
import os.path as osp
import albumentations as A
from time import time
from matplotlib import pyplot as plt
from glob import glob
import random
from albumentations.pytorch import ToTensorV2
ultralytics.checks()

model=YOLO("C:/Users/yj/OneDrive - UOS/바탕 화면/python workspace/intern_work/django/upd-v0.0.0/upd-v0.0.0/best.pt")

In [ ]:
default_transform = A.Compose([
        #A.ToGray(p=1),
        #A.CLAHE(always_apply=True),
        A.Resize(640,640)
        #A.Normalize(mean=0, std=1)
    ])

def Data_preprocessing_byA_list(image_path, transform=default_transform, crop_param=2):

  #device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

  image=cv2.imread(image_path)
  image=cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
  width, height,_ = image.shape
  crop_width = width // crop_param
  crop_height = height // crop_param
  parts=[]
  for i in range(crop_param):
      for j in range(crop_param):
          left = i * crop_width
          top = j * crop_height
          right = (i + 1) * crop_width
          bottom = (j + 1) * crop_height

          #cropped_image = image.crop((left, top, right, bottom))
          part = image[left:right,top:bottom,:]
          part=transform(image=part)['image']
          parts.append(part)
  #parts = torch.stack(parts)
  return parts

In [6]:
#폴더 yolo 형식으로 정리: train val test 나누고 -> images labels 폴더 따로따로 존재함
import splitfolders
from glob import glob
import os.path as osp
tgt_images_dir="./datasets/box/relabeling/images/"
tgt_labels_dir="./datasets/box/relabeling/labels/"

imgs=glob(osp.join(tgt_images_dir, '*'))
labels=glob(osp.join(tgt_labels_dir, '*'))
imgs.sort()
labels.sort()
if(len(imgs)!=len(labels)):
    print("안맞음")
else:
    splitfolders.ratio("./datasets/box/relabeling/", output="./datasets/box/relabeling/", seed=1337, ratio=(.8, .1, .1))
    
        


Copying files: 0 files [00:00, ? files/s]

Copying files: 4866 files [00:09, 517.60 files/s]


In [15]:
tgt_images_dir="./datasets/box/relabeling/valid/images"
tgt_labels_dir="./datasets/box/relabeling/valid/labels"

imgs=glob(osp.join(tgt_images_dir, '*'))
labels=glob(osp.join(tgt_labels_dir, '*'))

osp.basename

imgs.sort()
labels.sort()

for i in range(len(imgs)):
    if osp.basename(imgs[i][:-3])!=osp.basename(labels[i][:-3]):
        print(f"다름, {osp.basename(imgs[i][:-3])}, {osp.basename(labels[i][:-3])}")


In [13]:
imgs

['./datasets/box/relabeling/test/images\\1_00d23adf-WhatsApp_Image_2022-07-19_at_2-05-05_PM_jpeg.rf.75e59b8de01dcd164838536dd4db6dfd.jpg',
 './datasets/box/relabeling/test/images\\1_06ea6da4-97f2e99b-imagen189_jpg.rf.a9bf2030376681610cca2a69e0aa0a01.jpg',
 './datasets/box/relabeling/test/images\\1_0ac37331-WhatsApp_Image_2022-07-26_at_4-38-39_PM_jpeg.rf.8121d135016b8bf3ec6f80d4dea9acd1.jpg',
 './datasets/box/relabeling/test/images\\1_104_png_jpg.rf.594340a8cfc87b3897c0c345600d0601.jpg',
 './datasets/box/relabeling/test/images\\1_105_png.rf.5a9bf43c8cc44e0509ab99daa3591ca9.jpg',
 './datasets/box/relabeling/test/images\\1_108_png_jpg.rf.840dc13006088c3da1b7bad71760436b.jpg',
 './datasets/box/relabeling/test/images\\1_10_png_jpg.rf.af61cc3ede54f2e2aba74eb7065574d8.jpg',
 './datasets/box/relabeling/test/images\\1_112_png_jpg.rf.55ff0cb2d92a35ec2bb1202adefa2aca.jpg',
 './datasets/box/relabeling/test/images\\1_113_png_jpg.rf.96ef4b57764803f99182bbaade188791.jpg',
 './datasets/box/relabeling/

In [ ]:
#img_list=glob("./datasets/classification/sub_neg/*")
image_dir="./datasets/box/relabeling/images"
label_dir="./datasets/box/relabeling/labels"

def auto_labeling(target_dir, image_dir, label_dir):

    img_list=glob(os.path.join(target_dir, "*"))
    os.makedirs(image_dir,exist_ok=True)
    os.makedirs(label_dir,exist_ok=True)
    for img in img_list:
        _, file_ext= os.path.splitext(img)
        img_name=os.path.basename(img)
        file_ext = file_ext.lower()

        if not (file_ext == '.jpg' or file_ext == '.jpeg' or file_ext == '.png'):
            continue

        parts=Data_preprocessing_byA_list(img)
        results=model(parts, device='cpu')
        idx=0
        for result in results:
            if(result):
                idx+=1
                plt.imshow(result.plot())
                plt.show()
                
                cv2.imwrite(os.path.join(image_dir, f"{idx}_"+img_name),cv2.cvtColor(result.orig_img, cv2.COLOR_BGR2RGB))
                labels=result.boxes.xywhn.tolist()

                with open(os.path.join(label_dir, f"{idx}_"+img_name[:-3]+'txt'), 'w') as f:
                    for label in labels:
                        x,y,w,h=label
                        f.write(f"1 {x} {y} {w} {h}\n")


            


In [ ]:
target="./new-tesis-5"
dirs=os.listdir(target)
for dir in dirs:
    
    if not osp.isdir(osp.join(target, dir)):
        continue
    target_dir=osp.join(target,dir,'images')
    #print(imgs)
    auto_labeling(target_dir, image_dir, label_dir)
        
